## Lib imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
import statistics
# from math import pi
# from matplotlib.gridspec import GridSpec
# from dateutil.relativedelta import *
# import matplotlib.dates as mdates
# from matplotlib.dates import DateFormatter
import seaborn as sns
import scipy.stats
# from statsmodels.graphics.gofplots import qqplot
# import cv2
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import cross_val_score, cross_val_predict, RepeatedKFold 
from sklearn.linear_model import LinearRegression, LogisticRegression
# from collections import Counter
import copy
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import sklearn.metrics as me
# from sklearn import linear_model
from sklearn.metrics import classification_report
import pingouin as pg
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Files reading

In [2]:
!python3 '/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/main_generate_json_files.py' --all_tweets True

Starting script...
Removing existing file...
Ending script...


In [3]:
data = pd.read_json(r'/Users/leosilva/development/thesis/workspacePyCharm/TwitterDataMining/alldata.json')
data.head()

,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
id_str_twitter,23781146,15148309,927028057763995648,77648005,3429765093,10794662,1034514325644431361,529445600,12050562,62314491,18289937,172880784,18609547,2164986296,1026783544457273345,109604376,1040939562774790144,69734383,119897261
participant_id,1,2,3,None,4,5,6,None,7,8,None,9,10,11,12,13,14,15,16
name,Fe Rebelatto,Sommelier de Airbnb,✪ Felippe,Conference Distinguished Specialist,Paula Santana,Maharaja of the Legacies,Jessica,"var nickname = ""Riquettinha"";",Roberta Arcoverde,Gabriela,Thamara Andrade,felipe.ts ⚡,Juan Lopes,Rafael Dantas,iagor com i,T̷o̷m̷o̷y̷o̷ ̷ F̷u̷r̷a̷c̷ã̷o̷,Esdras Xavier,belle-🏴.dev || twitch.tv/bellesamways,Willian Justen
screen_name,rebelatto,Guilh_rm_,FelippeRegazio,NannoKa,psanrosa13,rponte,dev_jessi,Riquettinha,rla4,Gabrielathalita,thamyk,felipefialho_,juanplopes,RaffaelDantass,Iagor51,RafaelMansilha,esdras_xavieer,bellesamways,Willian_justen
location,"Rio de Janeiro, Brasil",somewhere in Brazil,,Ctba-PR Brasil,Brasil,"Ceará, Brazil",São Paulo - Brasil,"Belo Horizonte, Brasil","Sao Paulo, Brazil",Belo Horizonte,"Belo Horizonte, Brazil.",Dark Side of the Moon,"Rio de Janeiro, Brasil","São Paulo, Brasil",Pvh,"Palmas, Tocantins, Brasil","Jaraguá do Sul, Brasil",Curitiba,"Petrópolis, Brasil"


In [4]:
def create_tweets_df_per_user(data, id_user):

    tweets_pd = pd.DataFrame(data[id_user]['tweets'])
    tweets_pd.columns = ['id', 'id_str_twitter', "text", "created_at", 'favorite_count', 'retweet_count',
                         'lang',
                         "id_user", "vader_sentiment_analysis_score", "vader_sentiment_analysis_polarity",
                         "vader_sentiment_analysis_polarity_5_cat",
                         "oplexicon_sentiment_analysis_score", "oplexicon_sentiment_analysis_polarity",
                         "oplexicon_sentiment_analysis_polarity_5_cat",
                         "sentistrength_sentiment_analysis_score", "sentistrength_sentiment_analysis_polarity",
                         "sentistrength_sentiment_analysis_polarity_5_cat",
                         "sentilexpt_sentiment_analysis_score", "sentilexpt_sentiment_analysis_polarity",
                         "sentilexpt_sentiment_analysis_polarity_5_cat",
                         "liwc_sentiment_analysis_score", "liwc_sentiment_analysis_polarity",
                         "liwc_sentiment_analysis_polarity_5_cat",
                         "final_score", "final_score_ensemble", 
                         "final_polarity", "final_polarity_ensemble", 
                         'text_updated', 'is_retweet', 'retweet_updated']
 
    return tweets_pd

In [5]:
all_tweets = pd.DataFrame()
for c in data:
    id_user = c
    if data[id_user]['bigfive']:
        tweets = create_tweets_df_per_user(data, id_user)
        columns_to_delete = ['id', 'id_str_twitter', "text", 'favorite_count', 'retweet_count', 'lang',
                             #"vader_sentiment_analysis_score", 
                             "vader_sentiment_analysis_polarity",
                             "vader_sentiment_analysis_polarity_5_cat",
                             #"oplexicon_sentiment_analysis_score",
                             "oplexicon_sentiment_analysis_polarity",
                             "oplexicon_sentiment_analysis_polarity_5_cat",
                             #"sentistrength_sentiment_analysis_score",
                             "sentistrength_sentiment_analysis_polarity",
                             "sentistrength_sentiment_analysis_polarity_5_cat",
                             #"sentilexpt_sentiment_analysis_score",
                             "sentilexpt_sentiment_analysis_polarity",
                             "sentilexpt_sentiment_analysis_polarity_5_cat",
                             #"liwc_sentiment_analysis_score",
                             "liwc_sentiment_analysis_polarity",
                             "liwc_sentiment_analysis_polarity_5_cat",
                             'text_updated', 'is_retweet', 'retweet_updated']

        for c in columns_to_delete:
            del tweets[c]

        tweets['Participant'] = data[id_user]['participant_id']
        if all_tweets.empty:
            all_tweets = tweets
        else:
            all_tweets = all_tweets.append(tweets)

In [6]:
all_tweets.head()

,created_at,id_user,vader_sentiment_analysis_score,oplexicon_sentiment_analysis_score,sentistrength_sentiment_analysis_score,sentilexpt_sentiment_analysis_score,liwc_sentiment_analysis_score,final_score,final_score_ensemble,final_polarity,final_polarity_ensemble,Participant
0,2021-03-31T19:13:28,26,-0.4588,0.000000,0.00,0.00,0.0000,-0.091760,0.000,neg,neu,1
1,2021-03-31T12:28:56,26,0.0000,0.000000,0.00,0.00,-0.0667,-0.013340,0.000,neg,neu,1
2,2021-03-31T12:17:06,26,0.7734,0.458831,0.00,0.00,0.0196,0.250366,0.000,pos,neu,1
3,2021-03-31T11:46:04,26,-0.8720,0.458831,0.25,-0.25,0.0400,-0.074634,0.000,neg,neu,1
4,2021-03-30T17:58:24,26,-0.6808,0.000000,0.00,-0.25,0.0000,-0.186160,-0.125,neg,neg,1


In [7]:
count = 1
json_data = {
    'Participant': [],
    'Id User': [],
    'O': [],
    'C': [],
    'E': [],
    'A': [],
    'N': []
}

for d in data:
    if data[d]['bigfive']:
        b = pd.DataFrame(data[d]['bigfive'])
        b.columns = ['id', 'O', 'C', 'E', 'A', 'N', 'Id User']
        b['Participant'] = count
        del b['id']
        b = b[['Participant', 'Id User', 'O', 'C', 'E', 'A', 'N']]
        json_data['Participant'].append(b['Participant'][0])
        json_data['Id User'].append(b['Id User'][0])
        json_data['O'].append(b['O'][0])
        json_data['C'].append(b['C'][0])
        json_data['E'].append(b['E'][0])
        json_data['A'].append(b['A'][0])
        json_data['N'].append(b['N'][0])
        count += 1
        
bf = pd.DataFrame(json_data)

In [8]:
bf.head()

,Participant,Id User,O,C,E,A,N
0,1,26,37,43,36,24,26
1,2,27,41,43,34,22,23
2,3,28,43,46,31,24,15
3,4,30,17,33,33,22,39
4,5,31,22,33,31,20,8


In [9]:
df_dict = {'id_user' : [],
            'pos' : [],
            'neu' : [],
            'neg' : [],
            'por_pos' : [],
            'por_neu' : [],
            'por_neg' : [],
            'mean' : [],
            'O': [],
            'C': [],
            'E': [],
            'A': [],
            'N': []
          }

df_grouped = all_tweets.groupby(['Participant'])
for i in range(1,len(df_grouped)+1):
    t = all_tweets.query('Participant == {}'.format(i))
    bigfive = bf.query('Participant == {}'.format(i))
    vader_df = t[['final_score_ensemble', 'final_polarity_ensemble']].copy()
    qtd_pos = vader_df.query('final_polarity_ensemble == "pos"').count()
    qtd_neu = vader_df.query('final_polarity_ensemble == "neu"').count()
    qtd_neg = vader_df.query('final_polarity_ensemble == "neg"').count()
    qtd_total = vader_df['final_polarity_ensemble'].count()
    df_dict['id_user'].append(bigfive['Id User'].iloc[0])
#     print(type(bigfive['Id User'].iloc[0]))
    df_dict['pos'].append(qtd_pos.iloc[0])
    df_dict['neu'].append(qtd_neu.iloc[0])
    df_dict['neg'].append(qtd_neg.iloc[0])
    df_dict['por_pos'].append(((qtd_pos / qtd_total) * 100).iloc[0])
    df_dict['por_neu'].append(((qtd_neu / qtd_total) * 100).iloc[0])
    df_dict['por_neg'].append(((qtd_neg / qtd_total) * 100).iloc[0])
    df_dict['mean'].append(vader_df['final_score_ensemble'].mean())

    df_dict['O'].append(bigfive['O'].iloc[0])
    df_dict['C'].append(bigfive['C'].iloc[0])
    df_dict['E'].append(bigfive['E'].iloc[0])
    df_dict['A'].append(bigfive['A'].iloc[0])
    df_dict['N'].append(bigfive['N'].iloc[0])

df = pd.DataFrame(df_dict)

biglist = ['O', 'C', 'E', 'A', 'N']
for b in biglist:
    res = []
    for r in df[b]:
        if r <= 10:
            res.append(1)
        elif r > 10 and r < 21:
            res.append(2)
        elif r > 20 and r < 31:
            res.append(3)
        elif r > 30 and r < 41:
            res.append(4)
        elif r > 40:
            res.append(5)

    df['{}_Coded'.format(b)] = res
df

,id_user,pos,neu,neg,por_pos,por_neu,por_neg,mean,O,C,E,A,N,O_Coded,C_Coded,E_Coded,A_Coded,N_Coded
0,26,1408,1866,1764,27.947598,37.038507,35.013894,-0.022431,37,43,36,24,26,4,5,4,3,3
1,27,1367,1926,1255,30.057168,42.348285,27.594547,-0.000698,41,43,34,22,23,5,5,4,3,3
2,28,1968,2031,1933,33.175995,34.238031,32.585974,-0.007192,43,46,31,24,15,5,5,4,3,2
3,30,1719,832,620,54.210028,26.237780,19.552192,0.113446,17,33,33,22,39,2,4,4,3,4
4,31,2993,1443,1557,49.941599,24.078091,25.980310,0.055160,22,33,31,20,8,3,4,4,2,1
5,32,4386,2693,2306,46.734150,28.694726,24.571124,0.057806,39,49,39,28,15,4,5,4,3,2
6,34,693,459,393,44.854369,29.708738,25.436893,0.051225,35,35,32,26,16,4,4,4,3,2
7,35,895,975,725,34.489403,37.572254,27.938343,0.028354,29,45,34,35,29,3,5,4,4,3
8,37,3235,1892,1329,50.108426,29.306072,20.585502,0.066105,40,46,38,22,21,4,5,4,3,3
9,38,1646,1336,1726,34.961767,28.377230,36.661003,-0.017339,32,46,19,18,28,4,5,2,2,3


## CORRELATIONS

In [10]:
df_corr = df[['por_pos', 'por_neu', 'por_neg', 'O', 'C', 'E', 'A', 'N', 'O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']]
df_corr.rcorr(method='spearman', upper='pval', decimals=4, padjust=None, stars=False)

,por_pos,por_neu,por_neg,O,C,E,A,N,O_Coded,C_Coded,E_Coded,A_Coded,N_Coded
por_pos,-,0.0000,0.0061,0.6461,0.3571,0.6275,0.2172,0.6009,0.5271,0.0091,0.5931,0.1507,0.8498
por_neu,-0.8971,-,0.1373,0.4577,0.3250,0.8237,0.0613,0.5895,0.4378,0.0091,0.9206,0.0104,0.7564
por_neg,-0.6529,0.3882,-,0.9219,0.2489,0.655,0.7932,0.5895,0.6875,0.0210,0.4814,0.4594,0.8633
O,-0.1244,0.2,0.0267,-,0.0136,0.2654,0.6096,0.0316,0.0000,0.0988,0.3406,0.6508,0.1175
C,-0.2467,0.263,0.3061,0.6018,-,0.193,0.3163,0.4431,0.0468,0.0001,0.6738,0.5072,0.5072
E,0.1315,0.0606,-0.1211,0.2961,0.3433,-,0.1796,0.5119,0.6077,0.2006,0.0001,0.3391,0.6415
A,-0.3264,0.4778,-0.0712,0.1383,0.2676,0.3532,-,0.5336,0.5530,0.5118,0.1764,0.0000,0.4891
N,-0.1416,0.146,0.146,-0.5379,-0.2064,-0.177,-0.1682,-,0.0089,0.8714,0.1406,0.9025,0.0000
O_Coded,-0.1708,0.2088,0.1091,0.9368,0.5034,0.139,0.1604,-0.6297,-,0.2115,0.5051,0.5671,0.0389
C_Coded,-0.6289,0.6289,0.5704,0.4273,0.8128,0.3379,0.1771,0.044,0.3303,-,0.7406,0.4125,1.


In [11]:
columns_features = ['O', 'C', 'E', 'A', 'N']
df_features = pd.DataFrame(df[columns_features])
df_features

,O,C,E,A,N
0,37,43,36,24,26
1,41,43,34,22,23
2,43,46,31,24,15
3,17,33,33,22,39
4,22,33,31,20,8
5,39,49,39,28,15
6,35,35,32,26,16
7,29,45,34,35,29
8,40,46,38,22,21
9,32,46,19,18,28


In [12]:
target = pd.DataFrame(df[['por_pos', 'por_neu', 'por_neg']])
target

,por_pos,por_neu,por_neg
0,27.947598,37.038507,35.013894
1,30.057168,42.348285,27.594547
2,33.175995,34.238031,32.585974
3,54.210028,26.237780,19.552192
4,49.941599,24.078091,25.980310
5,46.734150,28.694726,24.571124
6,44.854369,29.708738,25.436893
7,34.489403,37.572254,27.938343
8,50.108426,29.306072,20.585502
9,34.961767,28.377230,36.661003


In [13]:
combs = [] # lista que guarda a combinacao

def combinations(target,data):
    for i in range(len(data)):
        new_target = copy.copy(target)
        new_data = copy.copy(data)
        new_target.append(data[i])
        new_data = data[i+1:]
        combs.append(new_target)
        combinations(new_target, new_data)

In [14]:
def perform_linear_regression(features, targets):
    for f in features:
        X = df_features[f]
        for t in targets:        
            y = target[t]
            X = sm.add_constant(X)

            # Note the difference in argument order
            model = sm.OLS(y, X).fit()
            predictions = model.predict(X) # make the predictions by the model
#             print(model.f_pvalue)
            # Print out the statistics
            if float(model.f_pvalue) <= 0.05:
                print(model.summary())
#                 with open('linear_regression_main_results_{}.txt'.format(t), 'a') as f:
#                     f.write(str(model.summary()))

In [21]:
# ---------------
# POSITIVE TWEETS
# ---------------

combs = [] # lista que guarda a combinacao
tgt = []
# features_list = ['O', 'C', 'E', 'A', 'N']
features_list = ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
df_features = pd.DataFrame(df[features_list])

combinations(tgt, features_list)

targets = ['por_neg']

perform_linear_regression(combs, targets)

                            OLS Regression Results                            
Dep. Variable:                por_neg   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     6.984
Date:                Tue, 10 Aug 2021   Prob (F-statistic):             0.0193
Time:                        19:05:06   Log-Likelihood:                -45.273
No. Observations:                  16   AIC:                             94.55
Df Residuals:                      14   BIC:                             96.09
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5312     11.132     -0.138      0.8

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


In [16]:
# ---------------
# NEUTRAL TWEETS
# ---------------

targets = ['por_neu']

perform_linear_regression(combs, targets)

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/s

                            OLS Regression Results                            
Dep. Variable:                por_neu   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                     3.688
Date:                Tue, 10 Aug 2021   Prob (F-statistic):             0.0386
Time:                        19:01:13   Log-Likelihood:                -47.868
No. Observations:                  16   AIC:                             105.7
Df Residuals:                      11   BIC:                             109.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.4986     17.033     -0.675      0.5

[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
                            OLS Regression Results                            
Dep. Variable:                por_neu   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     7.561
Date:                Tue, 10 Aug 2021   Prob (F-statistic):             0.0157
Time:                        19:01:13   Log-Likelihood:                -51.218
No. Observations:                  16   AIC:                             106.4
Df Residuals:                      14   BIC:                             108.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


In [17]:
# ---------------
# NEGATIVE TWEETS
# ---------------

targets = ['por_neg']

perform_linear_regression(combs, targets)

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


                            OLS Regression Results                            
Dep. Variable:                por_neg   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     6.984
Date:                Tue, 10 Aug 2021   Prob (F-statistic):             0.0193
Time:                        19:01:13   Log-Likelihood:                -45.273
No. Observations:                  16   AIC:                             94.55
Df Residuals:                      14   BIC:                             96.09
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5312     11.132     -0.138      0.8

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


In [18]:
# LINEAR REGRESSION -- WITH -- CONSTANT VALUE

X = df_features["O"]
y = target["por_pos"]
X = sm.add_constant(X)

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

KeyError: 'O'

In [ ]:
# LINEAR REGRESSION -- WITH -- CONSTANT VALUE

X = df_features["C"]
y = target["por_pos"]
X = sm.add_constant(X)

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# LINEAR REGRESSION -- WITHOUT -- CONSTANT VALUE

# MORE THAN ONE VARIABLE
X = df_features[["O", "C"]]
y = target["por_pos"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# LINEAR REGRESSION -- WITH -- CONSTANT VALUE

# MORE THAN ONE VARIABLE
X = df_features[["O", "C"]]
y = target["por_pos"]
X = sm.add_constant(X)

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
df

In [ ]:
columns_features = ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
df_features = pd.DataFrame(df[columns_features])
df_features

In [ ]:
print(np.std(df_features['O_Coded']))
print(np.std(df_features['C_Coded']))
print(np.std(df_features['E_Coded']))
print(np.std(df_features['A_Coded']))
print(np.std(df_features['N_Coded']))

In [ ]:
combs = []
tgt = []
features_list = ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']

combinations(tgt, features_list)

In [ ]:
# ---------------
# POSITIVE TWEETS
# ---------------

targets = ['por_pos']

perform_linear_regression(combs, targets)

In [ ]:
# ---------------
# NEUTRAL TWEETS
# ---------------

targets = ['por_neu']

perform_linear_regression(combs, targets)

In [ ]:
# ---------------
# NEGATIVE TWEETS
# ---------------

targets = ['por_neg']

perform_linear_regression(combs, targets)

In [ ]:
df

In [ ]:
#NORMALIZAR OS DADOS COM STANDARD SCALER PARA VER SE MELHORA O MODELO DE REGRESSAO E CORRELACAO

In [ ]:
# scaler = MinMaxScaler()
# df_scaled = pd.DataFrame(scaler.fit_transform(df[['O', 'C', 'E', 'A', 'N', 'O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']]))
# df_scaled.columns = ['O', 'C', 'E', 'A', 'N', 'O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
# # del df_scaled['id_user']
# # del df_scaled['mean']
# # del df_scaled['pos']
# # del df_scaled['neu']
# # del df_scaled['neg']
# # df_scaled = pd.concat([df[['por_neg', 'por_neu', 'por_pos']], df_scaled], axis=1)

# df_scaled

In [ ]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df))
df_scaled.columns = ['id_user','pos', 'neu', 'neg', 'por_pos', 'por_neu', 'por_neg',
                     'mean', 'O', 'C', 'E', 'A', 'N', 'O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
del df_scaled['id_user']
del df_scaled['mean']
del df_scaled['pos']
del df_scaled['neu']
del df_scaled['neg']
df_scaled

In [ ]:
df_scaled.rcorr(method='spearman', upper='pval', decimals=4, padjust=None, stars=True)

In [ ]:
df_scaled.rcorr(method='spearman', upper='pval', decimals=4, padjust=None, stars=False)

In [ ]:
df_corr = df[['por_neg', 'por_neu', 'por_pos', 'O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']]
df_corr.rcorr(method='spearman', upper='pval', decimals=4, padjust=None, stars=False)

In [ ]:
sns.lmplot(x="N_Coded", y="O_Coded", data=df_scaled)

In [ ]:
# ocean = ['O', 'C', 'E', 'A', 'N']
# rate = 25

# for b in ocean:
#     scores = df[b]
#     scores = scores.sort_values()
#     last_index = int((rate * len(o_score))/100)
#     scores = scores.reset_index()
#     min_score = scores.iloc[last_index][b]
#     print(b, min_score)

#     df_scaled['{}_min'.format(b)] = np.where(df[b] <= min_score, 1, 0)

# df_scaled

In [ ]:
# ocean = ['O', 'C', 'E', 'A', 'N']
# rate = 25

# for b in ocean:
#     scores = df[b]
#     scores = scores.sort_values()
#     last_index = int((rate * len(o_score))/100)
#     scores = scores.reset_index()
#     max_score = scores.iloc[len(scores) - last_index][b]
#     print(b, max_score)

#     df_scaled['{}_max'.format(b)] = np.where(df[b] >= max_score, 1, 0)

# df_scaled

In [ ]:
# LINEAR REGRESSION -- WITH -- CONSTANT VALUE

# MORE THAN ONE VARIABLE
X = df_features[["A_Coded", "C_Coded", "E_Coded"]]
y = target["por_pos"]
X = sm.add_constant(X)

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
pg.normality(df_scaled)

In [ ]:
c = 5 # here c can take any value between 0 to 5
df_scaled['O_Coded'] = df_scaled['O_Coded'].apply(lambda x: x**c)
df_scaled['C_Coded'] = df_scaled['C_Coded'].apply(lambda x: x**c)
df_scaled['E_Coded'] = df_scaled['E_Coded'].apply(lambda x: x**c)
df_scaled['A_Coded'] = df_scaled['A_Coded'].apply(lambda x: x**c)
df_scaled['N_Coded'] = df_scaled['N_Coded'].apply(lambda x: x**c)

In [ ]:
pg.normality(df_scaled)

In [ ]:
df_scaled['O_Coded'].hist()

In [ ]:
from sklearn.preprocessing import power_transform

df_scaled['O_Coded'] = power_transform(df_scaled[['O_Coded']], method='yeo-johnson')

In [ ]:
df_scaled['O_Coded'].hist()

In [ ]:
from sklearn.linear_model import LinearRegression

ind_var = 'C_Coded'

X = df_scaled[[ind_var]]
y = df_scaled["por_neg"]

# print(X)
# print(y)

plt.scatter(y,X[ind_var])
lr = LinearRegression(fit_intercept=False)

plt.plot(X[[ind_var]],lr.fit(X[[ind_var]], y).predict(X[[ind_var]]),color="k")

lr = LinearRegression()
plt.plot(X[[ind_var]],lr.fit(X[[ind_var]], y).predict(X[[ind_var]]),color="g")

plt.legend(['no intercept','with intercept'])

In [ ]:
df_scaled

In [ ]:
pg.normality(df_scaled)

In [ ]:
feature_cols = ['C_Coded', 'A_Coded', 'E_Coded']
X = df_scaled[feature_cols]
y = df_scaled['por_pos']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print('------------------------')
print('SKLEARN LINEARREGRESSION')
print('------------------------')
print()

print("Intercept : ", regressor.intercept_)
print("COEF: ", regressor.coef_)
print("Score: ", regressor.score(X_train, y_train))
print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))

print()
print('-----------------')
print('STATSMODELS OLS')
print('-----------------')

X_train = sm.add_constant(X_train)
model1 = sm.OLS(y_train,X_train)

result=model1.fit()

print(result.summary())

In [ ]:
feature_cols = ['C_Coded', 'A_Coded', 'E_Coded']
X = df_scaled[feature_cols]
y = df_scaled['por_neg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print('------------------------')
print('SKLEARN LINEARREGRESSION')
print('------------------------')
print()

print("Intercept : ", regressor.intercept_)
print("COEF: ", regressor.coef_)
print("Score: ", regressor.score(X_train, y_train))
print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))

print()
print('-----------------')
print('STATSMODELS OLS')
print('-----------------')

X_train = sm.add_constant(X_train)
model1 = sm.OLS(y_train,X_train)

result=model1.fit()

print(result.summary())

In [ ]:
feature_cols = ['C', 'A']
X = df_scaled[feature_cols]
y = df_scaled['por_neu']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

print('------------------------')
print('SKLEARN LINEARREGRESSION')
print('------------------------')
print()

print("Intercept : ", regressor.intercept_)
print("COEF: ", regressor.coef_)
print("Score: ", regressor.score(X_train, y_train))
print('Mean Absolute Error:', me.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', me.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(me.mean_squared_error(y_test, y_pred)))

print()
print('-----------------')
print('STATSMODELS OLS')
print('-----------------')

# print(me.SCORERS.keys())

X = sm.add_constant(X)
model1 = sm.OLS(y,X)

ols_cv = cross_val_score(regressor, X, y, cv=4)
print('R2: ', ols_cv)
ols_cv = cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=4)
print('NMSE: ', ols_cv)

result=model1.fit()

print(result.summary())

fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(result, 'C', fig=fig)

In [ ]:
columns_features = ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
df_features = pd.DataFrame(df_scaled[columns_features])
# print(df_features)

targets = ['por_neu']

perform_linear_regression(combs, targets)

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

# rng = np.random.RandomState(0)

feature_cols = ['C', 'A']
X = df_scaled[feature_cols]
y = df_scaled['por_neu']

sv = SVR(C=1.0, epsilon=0.2)

# regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
result = sv.fit(X, y)
print(sv.score(X,y))
print(sv.intercept_)

cvs = cross_val_score(sv, X, y, scoring='r2', cv=5)
print('R2: ', cvs)

In [ ]:
df

In [ ]:
df_scaled

In [ ]:
plt.scatter(df['C'], df['por_neg'])
plt.show()
plt.scatter(df['O'], df['por_neg'])
plt.show()
plt.scatter(df['E'], df['por_neg'])
plt.show()

In [ ]:
combs = [] # lista que guarda a combinacao

tgt = []
features_list = ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']

combinations(tgt, features_list)

def Sorting(lst):
    lst.sort(key=len)
    return lst

combs = Sorting(combs)

In [26]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

test_mse = []

for i in range(1,6):

    for c in combs:
        X = df[c]
        y = df['por_pos']
        print('COMBINATION: ', c)

        poly_reg = PolynomialFeatures(degree=i)
        X_poly = poly_reg.fit_transform(X)

        model = LinearRegression()
        model.fit(X_poly, y)

    #     cvs = cross_val_score(model, X_poly, y, scoring='r2', cv=5)
        y_poly_pred = model.predict(X_poly)
    #     print(np.mean(cvs))

        model = sm.OLS(y, sm.add_constant(X_poly)).fit()
        if model.f_pvalue <= 0.05:
    #         print(model.rsquared)
            print(model.summary())
            rmse = np.sqrt(me.mean_squared_error(y, y_poly_pred))
            test_mse.append(rmse)
            r2 = me.r2_score(y, y_poly_pred)
            print('RMSE: ', rmse)
            print('R2: ', r2)
    #         print('COEF: ', model.coef_)
    #         print('INTERCEPT: ', model.intercept_)
            print()
    #         print(model.score(X_poly, y))

    #         dfPoly = pd.DataFrame(
    #             data=X_poly, 
    #             columns=poly_reg.get_feature_names(X.columns))

    #         print(dfPoly)

print("Best fit polynomial degree: ", min(test_mse))

COMBINATION:  ['O_Coded']
COMBINATION:  ['O_Coded', 'C_Coded']
                            OLS Regression Results                            
Dep. Variable:                por_pos   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     6.160
Date:                Tue, 10 Aug 2021   Prob (F-statistic):             0.0131
Time:                        19:59:53   Log-Likelihood:                -54.429
No. Observations:                  16   AIC:                             114.9
Df Residuals:                      13   BIC:                             117.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


COMBINATION:  ['O_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'N_Coded']
COMBINATION:  ['C_Coded']
                            OLS Regression Results                            
Dep. Variable:                por_pos   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     13.27
Date:                Tue, 10 Aug 2021   Prob (F-statistic):            0.00266
Time:                        19:59:53   Log-Likelihood:                -54.429
No. Observations:                  16   AIC:                             112.9
Df Residuals:                      14   BIC:                             114.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/s

COMBINATION:  ['E_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['E_Coded', 'N_Coded']
COMBINATION:  ['A_Coded']
COMBINATION:  ['A_Coded', 'N_Coded']
COMBINATION:  ['N_Coded']
COMBINATION:  ['O_Coded']
COMBINATION:  ['O_Coded', 'C_Coded']
                            OLS Regression Results                            
Dep. Variable:                por_pos   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     4.106
Date:                Tue, 10 Aug 2021   Prob (F-statistic):             0.0282
Time:                        19:59:54   Log-Likelihood:                -52.453
No. Observations:                  16   AIC:                             114.9
Df Residuals:                      11   BIC:                             118.8
Df Model:                           4                                         
Covariance Type:            nonrobust               

...

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


 ['E_Coded']
COMBINATION:  ['E_Coded', 'A_Coded']
COMBINATION:  ['E_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['E_Coded', 'N_Coded']
COMBINATION:  ['A_Coded']
COMBINATION:  ['A_Coded', 'N_Coded']
COMBINATION:  ['N_Coded']
COMBINATION:  ['O_Coded']
COMBINATION:  ['O_Coded', 'C_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'A_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'E_Coded']
COMBINATION:  ['O_Coded', 'E_Coded', 'A_Coded']
COMBINATION:  ['O_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'E_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'A_Coded']
COMBINATION:  ['O_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'N_Coded']
COMBINAT

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


COMBINATION:  ['C_Coded', 'A_Coded']
COMBINATION:  ['C_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['C_Coded', 'N_Coded']
COMBINATION:  ['E_Coded']
COMBINATION:  ['E_Coded', 'A_Coded']
COMBINATION:  ['E_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['E_Coded', 'N_Coded']
COMBINATION:  ['A_Coded']
COMBINATION:  ['A_Coded', 'N_Coded']
COMBINATION:  ['N_Coded']
COMBINATION:  ['O_Coded']
COMBINATION:  ['O_Coded', 'C_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'E_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'A_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'C_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'E_Coded']
COMBINATION:  ['O_Coded', 'E_Coded', 'A_Coded']
COMBINATION:  ['O_Coded', 'E_Coded', 'A_Coded', 'N_Coded']
COMBINATION:  ['O_Coded', 'E_Coded', 'N_Cod

/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
/Users/leosilva/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1604: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))
